# Inverse Kinematics Optimization

The previous doc explained features and how they define objectives of a constrained optimization problem. Here we show how to use this to solve IK optimization problems.

At the bottom there is more general text explaining the basic concepts.

## Demo of features in Inverse Kinematics

Let's setup a standard configuration. (Lock the window with "Always on Top".)

In [2]:
import sys
sys.path.append('../build') #rai/lib')
import numpy as np
import libry as ry

**ry-c++-log** /home/student/Desktop/KOMO/rai-python/rai/rai/ry/ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback** INFO:/home/student/Desktop/KOMO/rai-python/rai/rai/Core/util.cpp:initCmdLine:602(1) ** cmd line arguments: 'rai-pybind -python'
** INFO:/home/student/Desktop/KOMO/rai-python/rai/rai/Core/util.cpp:initCmdLine:606(1) ** run path: '/home/student/Desktop/KOMO/rai-python/tutorials'
** INFO:/home/student/Desktop/KOMO/rai-python/rai/rai/Core/graph.cpp:initParameters:1363(1) ** parsed parameters:
{python,
LGP/cameraFocus:[1, 0.5, 1],
LGP/collisions:1,
LGP/stopTime:300,
LGP/stopSol:6,
opt/maxStep:0.1}



** INFO:/home/student/Desktop/KOMO/rai-python/rai/rai/ry/ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback


In [3]:
C = ry.Config()
C.addFile('../rai-robotModels/pr2/pr2.g')
C.addFile('../rai-robotModels/objects/kitchen.g')
C.view()

For simplicity, let's add a frame that represents goals

In [4]:
goal = C.addFrame("goal")
goal.setShape(ry.ST.sphere, [.05])
goal.setColor([.5,1,1])
goal.setPosition([1,.5,1])
X0 = C.getFrameState() #store the initial configuration

We create an IK engine. The only objective is that the `positionDiff` (position difference in world coordinates) between `pr2L` (the yellow blob in the left hand) and `goal` is equal to zero:

In [5]:
IK = C.komo_IK(False)
IK.addObjective(type=ry.OT.eq, times = [1,2], feature=ry.FS.positionDiff, frames=['pr2L', 'goal'])

We now call the optimizer (True means with random initialization/restart).

In [6]:
IK.optimize()
IK.getReport()

** KOMO::run solver:dense collisions:0 x-dim:25 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:25 #frames:358
** optimization time:0.00914103 (kin:0.000131 coll:0.000132 feat:0 newton: 0.00105) setJointStateCount:35
   sos:0.0808073 ineq:0 eq:0.238354


{'F_qItself/1-#46': {'order': 1.0, 'type': 'sos', 'sos': 0.0808072604798673},
 'F_qQuaternionNorms/0-#179': {'order': 0.0, 'type': 'eq'},
 'F_PositionDiff/0-pr2L-goal': {'order': 0.0,
  'type': 'eq',
  'eq': 0.23835408214217368},
 'sos': 0.0808072604798673,
 'ineq': 0.0,
 'eq': 0.23835408214217368,
 'f': 0.0}

The best way to retrieve the result is to copy the optimized IK configuration back into your working configuration C, which is now also displayed

In [7]:
#IK.getFrameState(1)
C.setFrameState(IK.getFrameState(0))

We can redo the optimization, but for a different configuration, namely a configuration where the goal is in another location.
For this we move goal in our working configuration C, then copy C back into the IK engine's configurations:

In [10]:
## (iterate executing this cell for different goal locations!)

# move goal
goal.setPosition([.8,.2,.5])

# copy C into the IK's internal configuration(s)
IK.setConfigurations(C)

# reoptimize
IK.optimize(0.) # 0: no adding of noise for a random restart
#print(IK.getReport())
print(np.shape(IK.getFrameState(0)))
print(np.shape(IK.getFrameState(0)[1]))
# grab result
# C.setFrameState( IK.getConfiguration(1) )
C.setFrameState(IK.getFrameState(0))

** KOMO::run solver:dense collisions:0 x-dim:25 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:25 #frames:358
** optimization time:0.000305789 (kin:0.000238 coll:0.000149 feat:0 newton: 0.001415) setJointStateCount:3
   sos:0.000285026 ineq:0 eq:0.0270084
(179, 7)
(7,)


Let's solve some other problems, always creating a novel IK engine:

The relative position of `goal` in `pr2R` coordinates equals [0,0,-.2] (which is 20cm straight in front of the yellow blob)

In [99]:
C.setFrameState(X0)
IK = C.komo_IK(False)
IK.addObjective(type=ry.OT.eq,times=[1], feature=ry.FS.positionRel, frames=['goal','pr2R'], target=[0,0,-.2])
IK.optimize()
C.setFrameState(IK.getFrameState(0))

** KOMO::run solver:dense collisions:0 x-dim:25 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:25 #frames:358
** optimization time:0.00105824 (kin:5.2e-05 coll:1.1e-05 feat:0 newton: 0.000124) setJointStateCount:12
   sos:0.00848536 ineq:0 eq:0.0341739


The distance between `pr2R` and `pr2L` is zero:

In [120]:
C.setFrameState(X0)
IK = C.komo_IK(False)
IK.addObjective(type=ry.OT.eq, times=[1], feature=ry.FS.distance, frames=['pr2L','pr2R'])
IK.optimize()
C.setFrameState(IK.getFrameState(0))

** KOMO::run solver:dense collisions:0 x-dim:25 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:25 #frames:358
** optimization time:0.00069327 (kin:3.3e-05 coll:5e-06 feat:0 newton: 5.9e-05) setJointStateCount:6
   sos:0.00209253 ineq:0 eq:0.0149894


The 3D difference between the z-vector of `pr2R` and the z-vector of `goal`:

In [125]:
C.setFrameState(X0)
IK = C.komo_IK(False)
IK.addObjective(type=ry.OT.eq, times=[1], feature=ry.FS.vectorZDiff, frames=['pr2R', 'goal'])
IK.optimize()
C.setFrameState(IK.getFrameState(0))

** KOMO::run solver:dense collisions:0 x-dim:25 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:25 #frames:358
** optimization time:0.00144349 (kin:0.000111 coll:2.9e-05 feat:0 newton: 0.000115) setJointStateCount:12
   sos:0.0163838 ineq:0 eq:0.0143332


The scalar product between the z-vector of `pr2R` and the z-vector of `goal` is zero:

In [126]:
C.setFrameState(X0)
IK = C.komo_IK(False)
IK.addObjective(type=ry.OT.eq, times=[1], feature=ry.FS.scalarProductZZ, frames=['pr2R', 'goal'])
IK.optimize()
C.setFrameState(IK.getFrameState(0))

** KOMO::run solver:dense collisions:0 x-dim:25 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:25 #frames:358
** optimization time:0.000686185 (kin:7.1e-05 coll:3e-06 feat:0 newton: 4.2e-05) setJointStateCount:4
   sos:0.000248896 ineq:0 eq:0.00308733


etc etc

## More explanations

All methods to compute paths or configurations solve constrained optimization problems. To use them, you need to learn to define constrained optimization problems. Some definitions:

* An objective defines either a sum-of-square cost term, or an equality constraint, or an inequality constraint in the optimization problem. An objective is defined by its type and its feature. The type can be `sos` (sum-of-squares), `eq`, or `ineq`, referring to the three cases.
* A feature is a (differentiable mapping) from the decision variable (the full path, or robot configuration) to a feature space. If the feature space is, e.g., 3-dimensional, this defines 3 sum-of-squares terms, or 3 inequality, or 3 equality constraints, one for each dimension. For instance, the feature can be the 3-dim robot hand position in the 15th time slice of a path optimization problem. If you put an equality constraint on this feature, then this adds 3 equality constraints to the overall path optimization problem.
* A feature is defined by the keyword for the feature map (e.g., `pos` for position), typically by a set of frame names that tell which objects we refer to (e.g., `pr2L` for the left hand of the pr2), optionally some modifiers (e.g., a scale or target, which linearly transform the feature map), and the set of configuration IDs or time slices the feature is to be computed from (e.g., `confs=[15]` if the feat is to be computed from the 15th time slice in a path optimization problem).
* In path optimization problems, we often want to add objectives for a whole time interval rather than for a single time slice or specific configuration. E.g., avoid collisions from start to end. When adding objectives to the optimization problem we can specify whole intervals, with `times=[1., 2.]`, so that the objective is added to each configuration in this time interval.
* Some features, especially velocity and acceleration, refer to a tuple of (consecutive) configurations. E.g., when you impose an acceleration feature, you need to specify `confs=[13, 14, 15]`. Or if you use `times=[1.,1.]`, the acceleration features is computed from the configuration that corresponds to time=1 and the two configurations *before*.
* All kinematic feature maps (that depend on only one configuration) can be modified to become a velocity or acceleration features. E.g., the position feature map can be modified to become a velocity or acceleration feature.
* The `sos`, `eq`, and `ineq` always refer to the feature map to be *zero*, e.g., constraining all features to be equal to zero with `eq`. This is natural for many features, esp. when they refer to differences (e.g. `posDiff` or `posRel`, which compute the relative position between two frames). However, when one aims to constrain the feature to a non-zero constant value, one can modify the objective with a `target` specification.
* Finally, all features can be rescaled with a `scale` specification. Rescaling changes the costs that arise from `sos` objectives. Rescaling also has significant impact on the convergence behavior for `eq` and `ineq` constraints. As a guide: scale constraints so that if they *would* be treated as squared penalties (squaredPenalty optim mode, to be made accessible) convergence to reasonable approximate solutions is efficient. Then the AugLag will also converge efficiently to precise constraints.

In [9]:
# Designing a cylinder grasp

In [1]:
D=0
C=0

In [2]:
import sys
sys.path.append('../build') #rai/lib')
import numpy as np
import libry as ry

**ry-c++-log** /home/student/Desktop/KOMO/rai-python/rai/rai/ry/ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback** INFO:/home/student/Desktop/KOMO/rai-python/rai/rai/Core/util.cpp:initCmdLine:602(1) ** cmd line arguments: 'rai-pybind -python'
** INFO:/home/student/Desktop/KOMO/rai-python/rai/rai/Core/util.cpp:initCmdLine:606(1) ** run path: '/home/student/Desktop/KOMO/rai-python/tutorials'
** INFO:/home/student/Desktop/KOMO/rai-python/rai/rai/Core/graph.cpp:initParameters:1363(1) ** parsed parameters:
{python,
LGP/cameraFocus:[1, 0.5, 1],
LGP/collisions:1,
LGP/stopTime:300,
LGP/stopSol:6,
opt/maxStep:0.1}



** INFO:/home/student/Desktop/KOMO/rai-python/rai/rai/ry/ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback


In [8]:
C = ry.Config()
C.addFile('../rai-robotModels/pr2/pr2.g')
C.addFile('../rai-robotModels/objects/kitchen.g')
C.view()

In [9]:
C.setJointState([.7], ["l_gripper_l_finger_joint"])
C.setJointState( C.getJointState() )

In [10]:
goal = C.addFrame("goal")
goal.setShape(ry.ST.cylinder, [0,0,.2, .03])
goal.setColor([.5,1,1])
goal.setPosition([1.81,.5,1])
X0 = C.getFrameState()

In [11]:
C.setFrameState(X0)
goal.setPosition([1.81,.5,1])
IK = C.komo_IK(False)
IK.addObjective(type=ry.OT.eq, times=[1],feature=ry.FS.positionDiff, frames=['pr2L', 'goal'], scale=[[1,0,0],[0,1,0]])
IK.addObjective(type=ry.OT.ineq, times=[1], feature=ry.FS.positionDiff, frames=['pr2L', 'goal'], scale=[[0,0,1]], target=[0,0,.0005])
IK.addObjective(type=ry.OT.ineq, times=[1], feature=ry.FS.positionDiff, frames=['pr2L', 'goal'], scale=[[0,0,-1]], target=[0,0,-.0005])
IK.addObjective(type=ry.OT.sos, times=[1], feature=ry.FS.scalarProductZZ, frames=['pr2L', 'goal'], scale=[0.1])
IK.addObjective(type=ry.OT.eq, times=[1], feature=ry.FS.scalarProductXZ, frames=['pr2L', 'goal'])
IK.optimize()
C.setFrameState(IK.getFrameState(0))
IK.getReport()

** KOMO::run solver:dense collisions:0 x-dim:25 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:25 #frames:358
** optimization time:0.00273175 (kin:0.000107 coll:2.6e-05 feat:0 newton: 0.00033) setJointStateCount:33
   sos:0.355635 ineq:0.0438169 eq:0.634637


{'F_qItself/1-#46': {'order': 1.0, 'type': 'sos', 'sos': 0.3555859516074508},
 'F_qQuaternionNorms/0-#179': {'order': 0.0, 'type': 'eq'},
 'F_PositionDiff/0-pr2L-goal': {'order': 0.0,
  'type': 'ineq',
  'ineq': 0.043816882963903125},
 'F_ScalarProduct/0-pr2L-goal': {'order': 0.0,
  'type': 'eq',
  'eq': 0.0002406496240726086},
 'sos': 0.35563526477129614,
 'ineq': 0.043816882963903125,
 'eq': 0.6346373897838342,
 'f': 0.0}

In [ ]:
IK.view()